In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Function to calculate dihedral angle
def calculate_dihedral(a, b, c, d):
    b1 = a - b
    b2 = c - b
    b3 = d - c
    
    b1 = b1 / np.linalg.norm(b1)
    b2 = b2 / np.linalg.norm(b2)
    b3 = b3 / np.linalg.norm(b3)

    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)

    x = np.dot(n1, n2)
    y = np.dot(np.cross(n1, n2), b2)
    angle = np.arctan2(y, x)
    
    return np.degrees(angle)

# Function to load RNA from PDB file
def load_rna_from_pdb(pdb_file):
    atom_coords = []
    atom_indices = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith('ATOM'):
                parts = line.split()
                atom_idx = int(parts[1])  # Atom serial number
                x = float(parts[6])
                y = float(parts[7])
                z = float(parts[8])
                atom_coords.append([x, y, z])
                atom_indices.append(atom_idx)
    return np.array(atom_coords), atom_indices

# Function to visualize the RNA molecule
def visualize_rna(coords, bonds):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c='r', marker='o')

    for bond in bonds:
        ax.plot([coords[bond[0], 0], coords[bond[1], 0]],
                [coords[bond[0], 1], coords[bond[1], 1]],
                [coords[bond[0], 2], coords[bond[1], 2]], 'b-')

    plt.show()

# Main function
def main():
    pdb_file = 'example.pdb'  # Replace with your PDB file path

    # Load RNA
    atom_coords, atom_indices = load_rna_from_pdb(pdb_file)

    # Define ribose dihedral angles (generalized)
    angles = []
    bonds = []

    for i in range(len(atom_coords) - 3):
        a1 = atom_coords[i]
        a2 = atom_coords[i + 1]
        a3 = atom_coords[i + 2]
        a4 = atom_coords[i + 3]

        angle = calculate_dihedral(a1, a2, a3, a4)
        angles.append((i, i + 1, i + 2, i + 3, angle))
        bonds.append((i, i + 1))  # Add bond connections for visualization

    # Create DataFrame for angles
    angle_df = pd.DataFrame(angles, columns=['Atom1', 'Atom2', 'Atom3', 'Atom4', 'Dihedral Angle (degrees)'])
    print(angle_df)

    # Visualize RNA
    visualize_rna(atom_coords, bonds)

if __name__ == '__main__':
    main()
